In [1]:
using Revise
using LowRankVortex
using TransportBasedInference
using LinearAlgebra
using Statistics
using PotentialFlow
import PotentialFlow.Plates: Plate, Points, Blobs
import PotentialFlow.Motions: reset_velocity!
import PotentialFlow.Elements
import PotentialFlow.Properties: @property
using JLD
using BenchmarkTools
using ProgressMeter
using Interpolations
using Distributions

In [2]:
using Plots
default(fontfamily = "Computer Modern",
        tickfont = font("Computer Modern", 9), 
        titlefont = font("Computer Modern", 14), 
        guidefont = font("Computer Modern", 12),
        grid = false)
pyplot()
using LaTeXStrings

### Routines for the plots

In [3]:
function routine_spectrum(Λ::Array{Float64,1})
    Λ = sort(abs.(Λ); rev = true)

    plt = plot(layout = grid(1,3), legend = false, margin = 5*Plots.px, size = (600, 300))

    scatter!(plt[1,1], collect(1:length(Λ)), Λ, 
          yscale = :log10, xlabel = L"i", ylabel = L"\lambda_i")
    scatter!(plt[1,2], collect(1:length(Λ)), cumsum(Λ)./sum(Λ),
           xlabel = L"i", ylabel = "Normalized cumulative energy")
    scatter!(plt[1,3], Λ[1:end-1] - Λ[2:end], 
          yscale = :log10, xlabel = L"i", ylabel = L"\lambda_i - \lambda_{i+1}")
    return plt
end

function routine_plotCx(state, Cx::Matrix{Float64}, rx::Int64, config::VortexConfig, X::StepRangeLen, Y::StepRangeLen; withvortices::Bool=true)
    
    U, S, _ = svd(Symmetric(Cx))
    source = state_to_lagrange(state, config)
    
    # Default julia colors
    cur_colors = theme_palette(:auto)
    
    nlines = rx ÷ 3 + 1
    if mod(rx, 3) == 0
        nlines -= 1
    end
    
    plt = plot(layout = grid(nlines, 3))
    
    for i = 1:rx
        idxlines = (i÷3) + 1
        idxcols  = i - 3*(i÷3) 
        if mod(i, 3) == 0
            idxlines -= 1
            idxcols = 3
        end
        
        if withvortices == true
            for j=1:config.Nv
                # Put circles to show strength change
                scatter!(plt[idxlines, idxcols], 
                      [state[(j-1)*3+1]],
                      [state[(j-1)*3+2]],
                      markersize = 50*abs.(U[3*j,i]), markerstrokecolor = cur_colors[i],
                      markeralpha = 1.0, 
                      markerstrokewidth = 3,
                      markercolor = :white, legend = false)
            end
#             plot!(plt[idxlines, idxcols], source, markersize = 12, markeralpha = 0.5, 
#                   color = cgrad(reverse(colormap("RdBu")[10:end-10])),
#                   clim = (-1.0, 1.0), label = ["Vortices" "Sources"], legend = false, colorbar = false)
            plot!(plt[idxlines, idxcols],  xlim = (-2.0, 2.0), xticks = -2.0:1.0:2.0, 
                  ylim = (0, 1.2*maximum(imag.(config.zs))))

            for j=1:config.Nv
                # Put arrows to indicate directions of change
                plot!(plt[idxlines, idxcols], 
                      [state[(j-1)*3+1], state[(j-1)*3+1] - U[(j-1)*3+1,i]],
                      [state[(j-1)*3+2], state[(j-1)*3+2] - U[(j-1)*3+2,i]], 
                      linewidth = 2, arrow=(:closed, 0.5), arrowsize = 0.5, color = cur_colors[i], legend = false)
                plot!(plt[idxlines, idxcols], 
                      [state[(j-1)*3+1], state[(j-1)*3+1] + U[(j-1)*3+1,i]],
                      [state[(j-1)*3+2], state[(j-1)*3+2] + U[(j-1)*3+2,i]], 
                      linewidth = 2, arrow=(:closed, 0.5), arrowsize = 0.5, color = cur_colors[i], legend = false)
            end
        end
    end
    plt
end

routine_plotCx (generic function with 1 method)

In [4]:
function routine_plot(state, config::VortexConfig)
    source = state_to_lagrange(state, config)
    plt = plot(layout = grid(1,2))
    
    zc = mean(getfield.(source[1], :z))
    plot!(plt[1,1], size = (1000, 400), xlabel = L"x", ylabel = L"y")
    plot!(plt[1,1], xlims = (-2.0, 10.0), 
                      ylim = (-2, 2))
    plot!(plt[1,1], source, ratio = 1.0, legend = false, markersize = 5, color = cgrad([:blue; :white; :red]))
    scatter!(plt[1,1], real.(config.ss), imag.(config.ss), markersize = 5, color = colorant"orangered2")
    
    plot!(plt[1,2], xlims = (real(zc) - 2.0, real(zc) + 2.0), 
                      ylim = (-2, 2))
    plot!(plt[1,2], source, ratio = 1.0, legend = false, markersize = 5, color = cgrad([:blue; :white; :red]))

end

routine_plot (generic function with 1 method)

In [5]:
"""
    vortex_patch!(vort,zc,Γ,radius,nring[,δ=0])

Create a circular patch of vortex blobs, returned in `vort`. The centroid of the patch is at `zc`, its strength
(circulation) is `Γ`, and its radius is `radius`. The patch consists of `nring` rings; if `nring = 1`, the patch
consists of only a single vortex blob at the centroid. Each blob is assigned radius `δ`, which is 0 by default.
"""
function vortex_patch!(vort,zc,Γ,radius,nring::Int;δ=0)
    Δr = radius/(nring-1/2)
    dΓ = Γ/(1+8*nring*(nring-1)/2)
    @show dΓ
    push!(vort,Vortex.Blob(zc,dΓ,δ))
    for ir in 1:nring-1
        nθ = 8*ir
        for j = 0:nθ-1
            push!(vort,Vortex.Blob(zc + ir*Δr*exp(im*2π*j/nθ),dΓ,δ))
        end
    end
    return vort
end

"""
    vortex_patch(zc,Γ,radius,nring[,δ=0]) -> Vector{Vortex.Blob}

Create a circular patch of vortex blobs. The centroid of the patch is at `zc`, its strength
(circulation) is `Γ`, and its radius is `radius`. The patch consists of `nring` rings; if `nring = 1`, the patch
consists of only a single vortex blob at the centroid. Each blob is assigned radius `δ`, which is 0 by default.
"""
vortex_patch(zc,Γ,radius,nring::Int;δ=0) = vortex_patch!(Vortex.Blob{Float64,Float64}[],zc,Γ,radius,nring,δ=δ)

vortex_patch

### Configuration setup

In [6]:
#
Δtgif = 0.5

# Pressure sensors
Δs = 0.5
sensors = complex.(collect(-1.5:Δs:11.0))
Ny = length(sensors)


rpatch = 0.5 # initial radius of the vortex patch
dpatch = 1.5 # initial distance between patch centroids
Γpatch = 6.0 # strength of patch.
Upatch = Γpatch/(2*π*dpatch)
Nring = 4  # number of rings in each patch.
# The uncertainty is set to a small fraction 10% of the Δr between two consecutive rings
σr =  0.1*rpatch/(Nring-1/2)
σΓ =  1e-2
# Δt = 0.01#0.005*π^2*d0^2/abs(Γ0) # set the time step


config_data = let Nv = 1+(8*(Nring-1)*Nring)÷2,
             U = complex(0.0),
             ss = sensors, Δt = 5e-3, δ = 5e-2,
             ϵX = 1e-4, ϵΓ = 1e-4,
             β = 1.0,
             ϵY = 1e-1
    VortexConfig(Nv, U, ss, Δt, δ, ϵX, ϵΓ, β, ϵY)
end

Nv = config_data.Nv

xgrid = range(-2, 10, length= 800)
ygrid = range(-2, 2, length=100)

t0 = 0.0
tf = 12.0
tspan = (t0, tf)

(0.0, 12.0)

In [7]:
blobsmean = vortex_patch(im*dpatch/2, Γpatch, rpatch, Nring, δ = config_data.δ)

zmean = getfield.(blobsmean, :z)
Γmean = getfield.(blobsmean, :S);

dΓ = 0.12244897959183673


In [8]:
Re = 300
gridRe = 3

idxCFD = Int64[]

ssCFD = -1.5:0.01:11
NCFD = length(ssCFD)

for (i, xi) in enumerate(ssCFD)
    if sum(xi .∈ config_data.ss) == 1
        push!(idxCFD, copy(i))
    elseif sum(xi .∈ config_data.ss) == 2
        error()
    end
end

@assert ssCFD[idxCFD] == config_data.ss "Error in the selected sensors"

fullpress = load("/media/mat/HDD/VortexPatch.jl/notebooks/data/pressure_vortex_patch_CFD_t_"*
    string(ceil(Int64, tspan[end]))*"_Re_"*string(ceil(Int64, Re))*
                 "_gridRe_"*string(ceil(Int64, gridRe))*"_rpatch_"*string(ceil(Int64, 100*rpatch))*
                 "_dpatch_"*string(ceil(Int64, 100*dpatch))*
                 "_Gpatch_"*string(ceil(Int64, 100*Γpatch))*".jld")["p"]

# fullpress = load("/media/mat/HDD/VortexPatch.jl/notebooks/data/"*
#     "pressure_vortex_patch_CFD__t_12Re_300_gridRe_3_rpatch_30_dpatch_200_Gpatch_600.jld")["p"]

# pressure_vortex_patch_CFD__t_12_Re_300_gridRe_3_rpatch_40_dpatch_100_Gpatch_600

# pressure_vortex_patch_CFD__t_12Re_300_gridRe_3_rpatch_30_dpatch_200_Gpatch_600

yt = fullpress[idxCFD, :]


ΔtCFD = 5e-3
data = SyntheticData(collect(0.0:ΔtCFD:12.0), ΔtCFD, zeros(1), zeros(1, length(collect(0.0:ΔtCFD:12.0))), yt)

SyntheticData([0.0, 0.005, 0.01, 0.015, 0.02, 0.025, 0.03, 0.035, 0.04, 0.045  …  11.955, 11.96, 11.965, 11.97, 11.975, 11.98, 11.985, 11.99, 11.995, 12.0], 0.005, [0.0], [0.0 0.0 … 0.0 0.0], [0.20666625031920313 0.20586312335318369 … 0.01270008482244937 0.012691520986275705; 0.1742425891635577 0.1739152849704368 … 0.014284036091272545 0.014273872098890915; … ; 0.010299914791860725 0.010299173780658437 … 0.06942864330844263 0.06951685503790611; 0.00948066569936639 0.009479346992547167 … 0.05526044098983389 0.05532486199339147])

### Setup the sequential filter

In [9]:
config = deepcopy(config_data)
ϵy = AdditiveInflation(Ny, zeros(Ny), config.ϵY)

AdditiveInflation(26, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [0.010000000000000002 0.0 … 0.0 0.0; 0.0 0.010000000000000002 … 0.0 0.0; … ; 0.0 0.0 … 0.010000000000000002 0.0; 0.0 0.0 … 0.0 0.010000000000000002], [0.1 0.0 … 0.0 0.0; 0.0 0.1 … 0.0 0.0; … ; 0.0 0.0 … 0.1 0.0; 0.0 0.0 … 0.0 0.1])

In [10]:
senkf = StochEnKF(x-> x, ϵy, config.Δt, config.Δt; isfiltered = true)

Stochastic EnKF  with filtered = true


In [11]:
Nelist = reverse([100; 80; 60; 50; 40; 30; 25])#; 20; 15; 12; 10])
Nrun = 50
Ny = length(config.ss)
Nx = 3*config.Nv

147

In [12]:
Lxy = 1.0
Lyy = 1.0

1.0

In [ ]:
for Ne in Nelist
    @show Ne

#     rmse_lowranktab_85 = zeros(Nrun)
#     rmse_lowranktab_95 = zeros(Nrun)
#     rmse_lowranktab_99 = zeros(Nrun)
    rmse_enkftab = zeros(Nrun)
    rmse_localized_enkftab = zeros(Nrun)

#     rx_lowranktab_85 = zeros(Int64, Nrun, ceil(Int64, tf/config.Δt))
#     ry_lowranktab_85 = zeros(Int64, Nrun, ceil(Int64, tf/config.Δt))

#     rx_lowranktab_95 = zeros(Int64, Nrun, ceil(Int64, tf/config.Δt))
#     ry_lowranktab_95 = zeros(Int64, Nrun, ceil(Int64, tf/config.Δt))

#     rx_lowranktab_99 = zeros(Int64, Nrun, ceil(Int64, tf/config.Δt))
#     ry_lowranktab_99 = zeros(Int64, Nrun, ceil(Int64, tf/config.Δt))
    
    for k=1:Nrun
        @show Ne, k
        # Generate the initial condition
        X0 = zeros(Ny+Nx, Ne)

        for l=1:Ne
            for i=1:config.Nv
                # Perturbed position 
                zi = zmean[i] + σr*randn()*exp(im*π*rand())
                X0[Ny + 3*(i-1) + 1, l] = real(zi)
                X0[Ny + 3*(i-1) + 2, l] = imag(zi)
                # Perturbed circulation
                Γi = Γmean[i] + σΓ*randn()
                X0[Ny + 3*(i-1) + 3, l] = Γi
            end
        end

        timeidx  = 1601:2401
        
        if Ne > 25

        # Run EnKF
        Xf, Xa = senkf_symmetric_vortexassim(senkf, deepcopy(X0), tspan, config, data; 
                                             withfreestream  = false, P = serial)

        rmse_enkftab[k] = 0.0


        # Compute the pressure over the time interval
        for i in timeidx
            @assert config.Δt == data.Δt 
            idxCFD = deepcopy(i)
            X_enkf = deepcopy(Xa[i])
            Y_enkf = zeros(NCFD, Ne)

            ϵx = RecipeInflation([config.ϵX; config.ϵΓ])
            ϵmul = MultiplicativeInflation(config.β)

            # Perform state inflation
            ϵmul(X_enkf, 1, Nx)
            ϵx(X_enkf, 0, Nx, config)
            for i=1:Ne
                Y_enkf[:,i] .= pressure(collect(ssCFD) .+ 0.0*im, 
                                   vcat(state_to_lagrange(X_enkf[:,i], config)...), 0.0) .+ config.ϵY*randn(NCFD)
            end

            q50_enkf = median(Y_enkf; dims = 2)[:,1]
            rmse_enkftab[k] += 1/length(timeidx)*deepcopy(norm(fullpress[:,idxCFD]-q50_enkf)/sqrt(length(ssCFD)))
        end

        save("/media/mat/HDD/LowRankVortex.jl/localization/datavortexpatch/metric_vortexpatch_enkf_Ne_"*string(Ne)*".jld", 
             "rmse", deepcopy(rmse_enkftab))
        end
        
        # Run localized sEnKF
        
        Xflocalized, Xalocalized = localized_senkf_symmetric_vortexassim(senkf, Lxy, Lyy, deepcopy(X0), tspan, 
                                                         config, data; withfreestream  = false, P = serial)

        rmse_localized_enkftab[k] = 0.0


        # Compute the pressure over the time interval
        for i in timeidx
            @assert config.Δt == data.Δt 
            idxCFD = deepcopy(i)
            X_localized = deepcopy(Xalocalized[i])
            Y_localized = zeros(NCFD, Ne)

            ϵx = RecipeInflation([config.ϵX; config.ϵΓ])
            ϵmul = MultiplicativeInflation(config.β)

            # Perform state inflation
            ϵmul(X_localized, 1, Nx)
            ϵx(X_localized, 0, Nx, config)
            for i=1:Ne
                Y_localized[:,i] .= pressure(collect(ssCFD) .+ 0.0*im, 
                                   vcat(state_to_lagrange(X_localized[:,i], config)...), 0.0) .+ config.ϵY*randn(NCFD)
            end

            q50_localized = median(Y_localized; dims = 2)[:,1]
            rmse_localized_enkftab[k] += 1/length(timeidx)*deepcopy(norm(fullpress[:,idxCFD]-q50_localized)/sqrt(length(ssCFD)))
        end

        save("/media/mat/HDD/LowRankVortex.jl/localization/datavortexpatch/metric_vortexpatch_"*
             "localized_enkf_Ne_"*string(Ne)*".jld", 
             "rmse", deepcopy(rmse_localized_enkftab))
        
        


        #############################
        # Run adaptive low rank EnKF with 0.85

#         Xflowrank_85, Xalowrank_85, rxhist_85, ryhist_85 = adaptive_symmetric_lowrankvortexassim(enkf, deepcopy(X0), 
#                tspan, config, data; rxdefault = nothing, rydefault = nothing, ratio = 0.85, P = serial, isadaptive = true)

#         rx_lowranktab_85[k,:] .= deepcopy(rxhist_85)
#         ry_lowranktab_85[k,:] .= deepcopy(ryhist_85)

#         rmse_lowranktab_85[k] = 0.0


#         # Compute the pressure over the time interval
#         for i in timeidx
#             @assert config.Δt == data.Δt 
#             idxCFD = deepcopy(i)
#             X_lowrank_85 = deepcopy(Xalowrank_85[i])
#             Y_lowrank_85 = zeros(NCFD, Ne)

#             ϵx = RecipeInflation([config.ϵX; config.ϵΓ])
#             ϵmul = MultiplicativeInflation(config.β)

#             # Perform state inflation
#             ϵmul(X_lowrank_85, 1, Nx)
#             ϵx(X_lowrank_85, 0, Nx, config)
#             for i=1:Ne
#                 Y_lowrank_85[:,i] .= pressure(collect(ssCFD) .+ 0.0*im, 
#                                    vcat(state_to_lagrange(X_lowrank_85[:,i], config)...), 0.0) .+ config.ϵY*randn(NCFD)
#             end

#             q50_lowrank_85 = median(Y_lowrank_85; dims = 2)[:,1]
#             rmse_lowranktab_85[k] += 1/length(timeidx)*deepcopy(norm(fullpress[:,idxCFD]-q50_lowrank_85)/sqrt(length(ssCFD)))
#         end


#         save("/media/mat/HDD/LowRankVortex.jl/localization/datavortexpatch/metric_vortexpatch_lowrank_Ne_"*string(Ne)*"_ratio_"*string(ceil(Int64, 100*0.85))*".jld", 
#              "rmse", deepcopy(rmse_lowranktab_85), 
#              "rxhist", deepcopy(rx_lowranktab_85), 
#              "ryhist", deepcopy(ry_lowranktab_85))

        #############################
        # Run adaptive low rank EnKF with 0.95

#         Xflowrank_95, Xalowrank_95, rxhist_95, ryhist_95 = adaptive_symmetric_lowrankvortexassim(enkf, deepcopy(X0), 
#                tspan, config, data; rxdefault = nothing, rydefault = nothing, ratio = 0.95, P = serial, isadaptive = true)

#         rx_lowranktab_95[k,:] .= deepcopy(rxhist_95)
#         ry_lowranktab_95[k,:] .= deepcopy(ryhist_95)

#         rmse_lowranktab_95[k] = 0.0


#         # Compute the pressure over the time interval
#         for i in timeidx
#             @assert config.Δt == data.Δt 
#             idxCFD = deepcopy(i)
#             X_lowrank_95 = deepcopy(Xalowrank_95[i])
#             Y_lowrank_95 = zeros(NCFD, Ne)

#             ϵx = RecipeInflation([config.ϵX; config.ϵΓ])
#             ϵmul = MultiplicativeInflation(config.β)

#             # Perform state inflation
#             ϵmul(X_lowrank_95, 1, Nx)
#             ϵx(X_lowrank_95, 0, Nx, config)
#             for i=1:Ne
#                 Y_lowrank_95[:,i] .= pressure(collect(ssCFD) .+ 0.0*im, 
#                                    vcat(state_to_lagrange(X_lowrank_95[:,i], config)...), 0.0) .+ config.ϵY*randn(NCFD)
#             end

#             q50_lowrank_95 = median(Y_lowrank_95; dims = 2)[:,1]
#             rmse_lowranktab_95[k] += 1/length(timeidx)*deepcopy(norm(fullpress[:,idxCFD]-q50_lowrank_95)/sqrt(length(ssCFD)))
#         end


#         save("/media/mat/HDD/LowRankVortex.jl/localization/datavortexpatch/metric_vortexpatch_lowrank_Ne_"*string(Ne)*"_ratio_"*string(ceil(Int64, 100*0.95))*".jld", 
#              "rmse", deepcopy(rmse_lowranktab_95), 
#              "rxhist", deepcopy(rx_lowranktab_95), 
#              "ryhist", deepcopy(ry_lowranktab_95))


#         #############################
#         # Run adaptive low rank EnKF with 0.99

#         Xflowrank_99, Xalowrank_99, rxhist_99, ryhist_99 = adaptive_symmetric_lowrankvortexassim(enkf, deepcopy(X0), 
#                tspan, config, data; rxdefault = nothing, rydefault = nothing, ratio = 0.99, P = serial, isadaptive = true)

#         rx_lowranktab_99[k,:] .= deepcopy(rxhist_99)
#         ry_lowranktab_99[k,:] .= deepcopy(ryhist_99)

#         rmse_lowranktab_99[k] = 0.0

#         # Compute the pressure over the time interval
#         for i in timeidx
#             @assert config.Δt == data.Δt 
#             idxCFD = deepcopy(i)
#             X_lowrank_99 = deepcopy(Xalowrank_99[i])
#             Y_lowrank_99 = zeros(NCFD, Ne)

#             ϵx = RecipeInflation([config.ϵX; config.ϵΓ])
#             ϵmul = MultiplicativeInflation(config.β)

#             # Perform state inflation
#             ϵmul(X_lowrank_99, 1, Nx)
#             ϵx(X_lowrank_99, 0, Nx, config)
#             for i=1:Ne
#                 Y_lowrank_99[:,i] .= pressure(collect(ssCFD) .+ 0.0*im, 
#                                    vcat(state_to_lagrange(X_lowrank_99[:,i], config)...), 0.0) .+ config.ϵY*randn(NCFD)
#             end

#             q50_lowrank_99 = median(Y_lowrank_99; dims = 2)[:,1]
#             rmse_lowranktab_99[k] += 1/length(timeidx)*deepcopy(norm(fullpress[:,idxCFD]-q50_lowrank_99)/sqrt(length(ssCFD)))
#         end


#         save("/media/mat/HDD/LowRankVortex.jl/localization/datavortexpatch/metric_vortexpatch_lowrank_Ne_"*string(Ne)*"_ratio_"*string(ceil(Int64, 100*0.99))*".jld", 
#              "rmse", deepcopy(rmse_lowranktab_99), 
#              "rxhist", deepcopy(rx_lowranktab_99), 
#              "ryhist", deepcopy(ry_lowranktab_99))
    end
end

Ne = 25
(Ne, k) = (25, 1)
(Ne, k) = (25, 2)
(Ne, k) = (25, 3)
(Ne, k) = (25, 4)
(Ne, k) = (25, 5)
(Ne, k) = (25, 6)
(Ne, k) = (25, 7)
(Ne, k) = (25, 8)
(Ne, k) = (25, 9)
(Ne, k) = (25, 10)
(Ne, k) = (25, 11)
(Ne, k) = (25, 12)
(Ne, k) = (25, 13)
(Ne, k) = (25, 14)
(Ne, k) = (25, 15)
(Ne, k) = (25, 16)
(Ne, k) = (25, 17)
(Ne, k) = (25, 18)
(Ne, k) = (25, 19)
(Ne, k) = (25, 20)
(Ne, k) = (25, 21)
(Ne, k) = (25, 22)
(Ne, k) = (25, 23)
(Ne, k) = (25, 24)
(Ne, k) = (25, 25)
(Ne, k) = (25, 26)
(Ne, k) = (25, 27)
(Ne, k) = (25, 28)
(Ne, k) = (25, 29)
(Ne, k) = (25, 30)
(Ne, k) = (25, 31)
(Ne, k) = (25, 32)
(Ne, k) = (25, 33)
(Ne, k) = (25, 34)
(Ne, k) = (25, 35)
(Ne, k) = (25, 36)
(Ne, k) = (25, 37)
(Ne, k) = (25, 38)
(Ne, k) = (25, 39)
(Ne, k) = (25, 40)
(Ne, k) = (25, 41)
(Ne, k) = (25, 42)
(Ne, k) = (25, 43)
(Ne, k) = (25, 44)
(Ne, k) = (25, 45)
(Ne, k) = (25, 46)
(Ne, k) = (25, 47)
(Ne, k) = (25, 48)
(Ne, k) = (25, 49)
(Ne, k) = (25, 50)
Ne = 30
(Ne, k) = (30, 1)
(Ne, k) = (30, 2)
(Ne, k)